In [1]:
!pip install tokenizers
!pip install torchdata

     |████████████████████████████████| 6.6 MB 14.4 MB/s 
     |████████████████████████████████| 47 kB 4.7 MB/s 
     |████████████████████████████████| 138 kB 22.3 MB/s 
     |████████████████████████████████| 127 kB 55.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.datasets import Multi30k
from torch.utils.data import DataLoader
import numpy as np
import math

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
from copy import copy

In [3]:
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))

f = open("parallelcorpus.txt", "a")

for i in train_iter:
  for x in [x.rstrip("\n") for x in i]:
    f.write(x)
    f.write(' ')

f.close()


In [4]:
MAX_LEN = 64
VOCAB_SIZE = 32768
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=["[UNK]", "[PAD]", "[BOS]", "[EOS]"])
tokenizer.pre_tokenizer = Whitespace()
tokenizer.train(['parallelcorpus.txt'], trainer)

tokenizer.enable_padding(pad_id=1, length=MAX_LEN)
tokenizer.post_processor = TemplateProcessing(
    single="[BOS] $A [EOS]",
    special_tokens=[
        ("[BOS]", tokenizer.token_to_id("[BOS]")),
        ("[EOS]", tokenizer.token_to_id("[EOS]")),
    ],
)

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_enc = tokenizer.encode(src_sample.rstrip("\n"))
        src_batch.append(torch.tensor(src_enc.ids))

        tgt_enc = tokenizer.encode(tgt_sample.rstrip("\n"))
        tgt_batch.append(torch.tensor(tgt_enc.ids))



    return torch.stack(src_batch), torch.stack(tgt_batch)

In [5]:
BATCH_SIZE = 16
train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
train_dataloader = DataLoader(train_iter, BATCH_SIZE, collate_fn=collate_fn)

In [6]:
class Embedding(nn.Module):
    def __init__(self, vocab_size, embedding_size, pad_mask):
        super(Embedding, self).__init__()
        self.emb = nn.Embedding(vocab_size, embedding_size, padding_idx=pad_mask)

    def forward(self, x):
        return self.emb(x)

In [7]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model=512, d_ff=2048, d_h=8, dropout=0.1):
        super(EncoderLayer, self).__init__()
        
        self.d_model = d_model
        self.d_h = d_h
        self.d_ff = d_ff
        self.d_k = self.d_v = int(d_model / d_h)
        self.dropout = nn.Dropout(dropout)

        self.concatLinear = nn.Linear(d_model, d_model, bias=False) # Linear Layer for the concatenated head
        self.normalize = nn.LayerNorm(d_model) # Normalizing Layer

        self.feed_forward = nn.Sequential( # Feed Forward Layer
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )

        self.linears = nn.ModuleList([nn.Linear(d_model, self.d_k, bias=False) for _ in range(d_h*3)])

    def forward(self, x):
        heads = []
        for i in range(self.d_h):
            Q = self.linears[3*i](x) # Query Matrix
            K = self.linears[3*i + 1](x) # Key Matrix
            V = self.linears[3*i + 2](x) # Value Matrix

            scaledMatMul = Q @ K.transpose(-1, -2) / math.sqrt(self.d_k) # MatMul of Q and K -> Scale

            head = F.softmax(scaledMatMul) @ V # SoftMax -> MatMul of Q,K and V

            heads.append(head) # A Single Head

        Z = self.concatLinear(torch.cat((heads), -1)) # Concatenated heads -> Linear Layer

        Z = self.dropout(Z)

        AddNorm = self.normalize(x + Z) # Output of the First Add&Norm Layer
        
        Z = self.normalize(self.feed_forward(AddNorm) + AddNorm) # 1st Add&Norm -> Feed Forward -> 2nd Add&Norm

        return Z

In [50]:
class EncoderStack(nn.Module):
    def __init__(self, d_model, d_ff, d_h, dropout, N):
        super(EncoderStack, self).__init__()
        self.encoders = nn.ModuleList([EncoderLayer(d_model, d_ff, d_h, dropout) for _ in range(N)]) # Stacking Encoder Layer N Times

    def forward(self, x):
        for encoder in self.encoders:
            x = encoder(x)
        return x

In [51]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model=512, d_ff=2048, d_h=8, dropout=0.1): # parameters={}
        super(DecoderLayer, self).__init__()

        self.d_model = d_model
        self.d_h = d_h
        self.d_k = self.d_v = int(self.d_model / self.d_h)

        self.linears = nn.ModuleList([nn.Linear(d_model, self.d_k, bias=False) for _ in range(d_h*3)]) # Linear Layers

        self.firstLinear = nn.Linear(d_h * self.d_v, d_model, bias=False) # Linear Layer for the Concatenated Head

        self.secondLinear = nn.Linear(d_h * self.d_model, d_model, bias=False) # Linear Layer for the Concatenated Head(second multi-head attention)

        self.normalize = nn.LayerNorm(d_model)

        self.mask = torch.triu( # Lower Triangular Mask Matrix
            torch.tensor([[[-np.inf for _ in range(self.d_k)] for _ in range(self.d_k)] for _ in range(16)]), diagonal=1
        )

        self.feed_forward = nn.Sequential( # Feed Forward Layer
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )

    def forward(self, x, y):
        heads1 = []
        heads2 = []
        for i in range(self.d_h):

            # FIRST ATTENTION LAYER OF THE DECODER
            ''' Same as decoder, but here we have tgt(target) as the decoder's input. '''
            Q = self.linears[3*i](y) # Query Matrix
            K = self.linears[3*i+1](y) # Key Matrix
            V = self.linears[3*i+2](y) # Value Matrix
            
            scaledMatMul = Q @ K.transpose(-1, -2) / math.sqrt(self.d_k) # Matrix Multiplication of Q and K -> Scale
            maskedMat = scaledMatMul + self.mask # Masking, optional
            soft = F.softmax(scaledMatMul) # SoftMax

            head =  soft @ V # Matrix Multiplication of Scaled and Soft Maxed Q, K Matrices with V

            heads1.append(head) # Appending a Single Head

        Z1 = self.firstLinear(torch.cat((heads1), dim=-1)) # Concatenated Heads of the First Attention Layer

        AddNorm1 = self.normalize(x + Z1) # First Normalizing Layer

        for i in range(self.d_h): # Second Attention Layer

            # SECOND ATTENTION LAYER OF THE DECODER
            ''' A typical Attention layer, however, instead of Q and K matrices, we use the output of the encoder. '''
            scaledMat = x @ x.transpose(-1, -2) / math.sqrt(self.d_k) # Matrix Multiplication of SRC and it's Transpose -> Scale

            soft = F.softmax(scaledMat) # Soft Max

            head = soft @ Z1 # Matrix Multiplication of Scaled and Soft Maxed X, X.T and the Output of the Previous Multi Head Attention Layer

            heads2.append(head) # Appending a Single Head

        Z2 = self.secondLinear(torch.cat((heads2), dim=-1)) # Concatenated Heads of the Second Attention Layer

        AddNorm2 = self.normalize(AddNorm1 + Z2) # Second Normalizing Layer -- The Output of the First Normalizing Layer is Being Used

        Z = self.normalize(AddNorm2 + self.feed_forward(AddNorm2)) # Feed Forward -> Normalize

        return Z

In [52]:
class DecoderStack(nn.Module):
    def __init__(self, d_model, d_ff, d_h, dropout, N):
        super(DecoderStack, self).__init__()
        self.decoders = nn.ModuleList([DecoderLayer(d_model, d_ff, d_h) for _ in range(N)]) # Stacking Decoder Layer N Times

    def forward(self, x, y):
        for decoder in self.decoders:
            y = decoder(x, y)
        return y

In [53]:
class Transformer(nn.Module):
    def __init__(self, embedding_size=512, d_model=512, d_h = 8, d_ff=2048, vocab_size=32768, dropout=0.1, num_coder_layers=6):
        super(Transformer, self).__init__()
        self.d_model = d_model
        self.d_h = d_h
        self.linear = nn.Linear(d_model, vocab_size, bias=False) # Final Linear Layer of Our Model
        self.dropoutEnc = nn.Dropout(dropout)
        self.dropoutDec = nn.Dropout(dropout)
        self.softmax = F.softmax
        self.embed = Embedding(vocab_size, embedding_size, pad_mask=1)
        self.encoderStack = EncoderStack(d_model, d_ff, d_h, dropout, num_coder_layers)
        self.decoderStack = DecoderStack(d_model, d_ff, d_h, dropout, num_coder_layers)

    def PositionalEncoding(self, seq_len, d_model, n):
        P = torch.zeros(seq_len, d_model)
        for k in range(seq_len):
            for i in range(int(d_model/2)):
                denominator = math.pow(n, 2*i/d_model)
                P[k, 2*i] = math.sin(k/denominator)
                P[k, 2*i+1] = math.cos(k/denominator)
        return P

    def forward(self, x, y):
        x = (self.embed(x))

        y = (self.embed(y))

        pos_encoding = self.PositionalEncoding(self.d_model//self.d_h, self.d_model, 10000)

        x = pos_encoding + x

        y = pos_encoding + y

        x = self.dropoutEnc(x)

        y = self.dropoutDec(y)

        encoderOutput = self.encoderStack(x.float())

        decoderOutput = self.decoderStack(encoderOutput, y.float())

        logits = self.linear(decoderOutput)

        probs = self.softmax(logits)

        return probs

In [54]:
model = Transformer(embedding_size=512, d_model=512, d_h=8, d_ff=2048, vocab_size=32768, dropout=0.1, num_coder_layers=6)

In [55]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
running_loss = 0.0
i=0
for epoch in range(2):
    for source, target in train_dataloader:
        if i==1812:
            break
        input, labels = source, target
        optimizer.zero_grad()

        unchangedLabels = labels.detach().clone()

        outputs = torch.argmax(model(input, labels), dim=2)

        outputs = outputs.reshape(1, 1024)

        unchangedLabels = unchangedLabels.reshape(1, 1024)

        loss = criterion(torch.tensor(outputs, dtype=torch.float, requires_grad=True), torch.tensor(unchangedLabels, dtype=torch.float, requires_grad=True))

        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        i=i+1
        if i % 10 == 1:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss}')
            running_loss = 0.0
print("Finished Training")